# Imports:

In [1]:
import pandas as pd
import numpy as np

# Read the input traning and test data

In [2]:
test_df = pd.read_csv("./car/test.csv", header=None)
train_df = pd.read_csv("./car/train.csv", header=None)

In [3]:
print("train summary:")
print(train_df.describe())
print("===============")
print("test summary:")
print(test_df.describe())

train summary:
            0     1     2     3     4     5      6
count    1000  1000  1000  1000  1000  1000   1000
unique      4     4     4     3     3     3      4
top     vhigh  high     3     4   big   med  unacc
freq      259   255   253   337   341   344    698
test summary:
          0      1      2     3    4     5      6
count   728    728    728   728  728   728    728
unique    4      4      4     3    3     3      4
top     low  vhigh  5more  more  med  high  unacc
freq    190    188    190   248  247   249    512


In [23]:
sample = test_df.sample(20)
print(sample)

         0      1      2     3      4     5      6
705  vhigh  vhigh      3     2    med   med  unacc
166    med  vhigh  5more     4  small   med  unacc
265   high    med      4     2  small   low  unacc
391    med   high      3     2    big   low  unacc
160    low    low      4  more  small  high   good
625   high   high      4  more  small   med  unacc
499   high   high  5more  more    big   low  unacc
225    med    med      4     2  small  high  unacc
311    med  vhigh      3     2    med   med  unacc
299  vhigh    med  5more  more  small   med  unacc
634  vhigh  vhigh      2     2    med   med  unacc
147    med  vhigh      2     4    big   low  unacc
251  vhigh    med  5more     2    big   med  unacc
146    low   high      2  more    med   med    acc
668   high    low  5more     4  small  high    acc
467  vhigh   high      4  more  small   low  unacc
206    low   high  5more     4    big   med    acc
439   high    low      4     4  small   med  unacc
257    low    low      2     2 

In [24]:
# calculates the information gain
def entropy(S, Attribute_col_index, Label_col_index):
    print("entropy")

# calculates the gini index
def gini():
    print("gini")
    
# calculates the majority error
def majority_error(S,Label_col_index, attrib_idx):
#     print("Calculating Majority Error ...")
    freq = S.groupby(Label_col_index)[Label_col_index].count()
#     print(freq)
    ME_S = (freq.sum()- freq.max())/ freq.sum()
#     print("ME(S)=" + str(ME_S))
#     test = S.groupby([attrib_idx,Label_col_index])[Label_col_index].count()
#     print(test)
    attrib_label_grouped = S.groupby([attrib_idx,Label_col_index],as_index=False)[Label_col_index].count()
#     print(attrib_label_grouped)
#     print(attrib_label_grouped.groupby(attrib_idx).apply(lambda grp: (grp.sum()-grp.max())))
#     print(attrib_label_grouped.groupby(attrib_idx).apply(lambda grp: (grp.sum()-grp.max())/grp.sum()))
#     print(attrib_label_grouped.groupby(attrib_idx).apply(lambda grp: (grp.sum()/freq.sum())))
#     print(attrib_label_grouped.groupby(attrib_idx).apply(lambda grp: (grp.sum()/freq.sum())*(grp.sum()-grp.max())/grp.sum()))
    Expected_ME_of_attrib_values = attrib_label_grouped.groupby(attrib_idx).apply(lambda grp: (grp.sum()/freq.sum())*(grp.sum()-grp.max())/grp.sum()).sum()
#     print("Expected_ME_of_attrib_values=",Expected_ME_of_attrib_values)
    gain = ME_S - Expected_ME_of_attrib_values
    print("gain of attrib index ", attrib_idx, "=", gain[Label_col_index])
    print("-------")
    return gain[Label_col_index]

# returns the column index of the best splitter attribute
# S: set of examples
# Attributes: list of attributes to be evaluated
# splitter_algorithm: the splitter algorithm, can be one of the 3 values ("ME":Majority Error, "GI":Gini Index, "EN":Entropy)
def Best_spliter_attribute(S, Attributes, Label_col_index, splitter_algorithm):
    print("splitter algorithm:"  + splitter_algorithm)
    if len(Attributes) < 2:
        return Attributes[0]
    best_gain = 0
    best_attribute = Attributes[0]
    for v in Attributes:
        if v != Label_col_index:
            print("finding gain for attribute with col-idx=",str(v))
            gain_v = 0
            if splitter_algorithm == "EN":
                gain_v = entropy(S,Label_col_index, v)
            elif splitter_algorithm == "ME":
                print("using ME algorithm")
                gain_v = majority_error(S,Label_col_index,v)
            elif splitter_algorithm == "GI":
                gain_v = gini(S,Label_col_index,v)
            else:
                assert False, "Unknown splitter_algorithm:" + splitter_algorithm + "!!!"
            if gain_v > best_gain:
                best_gain = gain_v
                best_attribute = v
    print("best attrib is:",best_attribute)
    return best_attribute

atrib_name = {0:"buying", 1:"maint",2:"doors",3:"persons",4:"lug_boot",5:"safety"}

def predict(root, entry):
    example = {} 
    for i in range(6):
        example[atrib_name[i]] = entry[i]
    print(example)
    #{'buying': 'low', 'maint': 'high', 'doors': '2', 'persons': 'more', 'lug_boot': 'small', 'safety': 'low'}
    return predic_helper(root, example)

def predic_helper(root, example):
    root_attrib_name = root[0]
    example_attrib_val = example[root_attrib_name]
    if isinstance(root[1][example_attrib_val], list): # if attrib-node
        return predic_helper(root[1][example_attrib_val], example)
    else: # if leaf node
        return root[1][example_attrib_val]
    

In [30]:
# ##############              ID3 implementation:
# Input:
# S: the set of Examples
# Attributes: the set of measured attributes
# Label_col_index: column index of the target attribute (the prediction)
# max_tree_level: bounds the height of the tree
# splitter_algorithm: can be one of the 3 values ("ME":Majority Error, "GI":Gini Index, "EN":Entropy)
def ID3(S, Attributes, Label_col_index, max_tree_level, splitter_algorithm):
    if S[Label_col_index].nunique() == 1:                                               # if all examples have same label:   
        return S[Label_col_index].mode()[0]
    elif len(Attributes) == 0:                                                          # if Attributes empty
        return S[Label_col_index].mode()[0]
    else:
        # 1. Create a Root node for tree
        Root = [] # each "attribute node" is a list s.t. 
                                                    # 1st element = string attribute name
                                                    # 2nd element = dictionary children;
                                                            # key = each possible attribute value v
                                                            # value = an "attribute node" list;  or a string label for leaf nodes
        # 2. A = attribute in Attributes that best splits S
        A = Best_spliter_attribute(S, Attributes, Label_col_index, splitter_algorithm)
        Root.append(atrib_name[A]) # 1st element = string attribute name
        Root.append({})            # 2nd element = dictionary children;
        # 3. for each possible value v of that A can take:
        for v in S[A].unique(): # TODO: S[A].unique() might not include all possible value 
            # 1. Add a new tree branch corresponding to A=v
            # 2. Let Sv be the subset of examples in S with A=v
            Sv = S.loc[S[A] == v]
            if len(Sv) == 0:
                Root[1][v] = S[Label_col_index].mode()[0] # string label
            else:
                Attrib_minus_A = Attributes
                if len(Attrib_minus_A) > 0 and A in Attrib_minus_A:
                    Attrib_minus_A.remove(A)
                Root[1][v] = ID3(Sv, Attrib_minus_A,Label_col_index, max_tree_level,splitter_algorithm) # an "attribute node" list;
        return Root
        
######            main
Attributes = [0,1,2,3,4,5] # initially put all attributes except the label in Attributes set
tree = ID3(test_df, Attributes,6, 10, "ME")
print(tree)
print("#######")
entry = ["low","high" ,     "2",  "more"  ,  "med"  , "med" ,   "acc"]
print(entry, "label is", predict(tree, entry))

splitter algorithm:ME
finding gain for attribute with col-idx= 0
using ME algorithm
gain of attrib index  0 = 0.0
-------
finding gain for attribute with col-idx= 1
using ME algorithm
gain of attrib index  1 = 0.0
-------
finding gain for attribute with col-idx= 2
using ME algorithm
gain of attrib index  2 = 0.0
-------
finding gain for attribute with col-idx= 3
using ME algorithm
gain of attrib index  3 = -5.551115123125783e-17
-------
finding gain for attribute with col-idx= 4
using ME algorithm
gain of attrib index  4 = -5.551115123125783e-17
-------
finding gain for attribute with col-idx= 5
using ME algorithm
gain of attrib index  5 = 0.0
-------
best attrib is: 0
splitter algorithm:ME
finding gain for attribute with col-idx= 1
using ME algorithm
gain of attrib index  1 = 0.0
-------
finding gain for attribute with col-idx= 2
using ME algorithm
gain of attrib index  2 = 0.0
-------
finding gain for attribute with col-idx= 3
using ME algorithm
gain of attrib index  3 = 0.0
-------
